#Librerias

In [125]:
# used for manipulating directory paths
import os
import random

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize


# tells matplotlib to embed plots within the notebook
%matplotlib inline

#Importar datos

In [126]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [127]:
#  datos de entrenamiento almacenados en los arreglos X, y
data = np.loadtxt("/content/gdrive/MyDrive/dataset_2p/ecoli.csv", delimiter=',')
# print(data)



#Generar datos
Se mezclo los datos data para generar un dataset desordenado con el cual se pueda trabajar.

In [128]:
#Mezclar bien data
np.random.shuffle(data)
print(data)
y = data[:, 7]
y[y == 1] = 0
y[y == 2] = 1
y[y == 3] = 2
y[y == 4] = 3
y[y == 5] = 4
y[y == 6] = 5
y[y == 7] = 6
y[y == 8] = 7

#CREAR X y Y de prueba
xp = data[300:, :7]
yp = y[300:, ]
print(xp.shape)

xp = xp.astype(float)

#CREAR X y Y normales
x = data[:300, :7]
y = y[:300,]


x = x.astype(float)
y = y.astype(int)
y = np.squeeze(y)
print(y)



#m = y.size

[[0.75 0.37 0.48 ... 0.7  0.74 4.  ]
 [0.27 0.42 0.48 ... 0.38 0.43 1.  ]
 [0.63 1.   0.48 ... 0.51 0.49 3.  ]
 ...
 [0.44 0.28 0.48 ... 0.27 0.37 1.  ]
 [0.69 0.65 0.48 ... 0.48 0.41 3.  ]
 [0.72 0.46 0.48 ... 0.66 0.7  2.  ]]
(36, 7)
[3 0 2 3 1 1 0 0 4 0 0 0 0 1 0 1 2 1 0 6 0 0 2 1 1 0 2 2 3 4 1 0 4 0 0 4 3
 2 0 0 1 3 0 0 0 1 2 1 3 0 0 0 0 0 3 3 4 1 2 3 0 0 0 1 0 1 1 0 2 0 3 0 7 5
 0 2 0 3 1 0 0 0 0 0 0 0 0 1 0 3 2 1 3 0 1 3 2 2 1 1 1 0 0 3 0 2 1 0 2 1 1
 0 0 3 2 0 7 2 1 4 0 2 4 2 0 0 0 0 2 1 4 0 2 2 4 2 0 1 0 0 1 0 3 4 1 0 1 3
 3 0 0 1 0 5 0 0 2 0 1 2 0 0 0 0 4 1 2 1 0 1 2 0 1 1 4 2 0 0 3 0 1 0 1 0 1
 1 0 0 4 3 0 0 0 1 1 0 6 2 0 1 1 0 0 0 1 5 1 2 1 2 4 2 3 2 2 0 3 2 0 4 0 0
 0 1 1 1 2 1 1 1 0 0 2 0 1 0 0 0 1 0 0 0 3 0 1 5 3 0 2 1 0 3 1 0 0 0 4 0 1
 1 4 3 2 4 2 1 0 1 0 0 3 1 2 0 2 0 1 0 2 0 0 2 1 0 1 0 2 2 0 0 0 1 4 5 0 1
 0 1 1 0]


#Determinar parametros
Se determinan parametros, pesos

In [129]:
# Configurando parametros necesario
input_layer_size  = 7  # Entrada de 13 caracteristicas
hidden_layer_size = 9   # 5 unidades ocultas
num_labels = 8        # 3 etiquetas, de 0 a 2

x1 = input_layer_size
h1 = hidden_layer_size
out = num_labels

#Pesos (Se puede aumentar)
pesos = {}
pesos['Theta1'] = np.random.rand(h1, x1 + 1)
pesos['Theta2'] = np.random.rand(out, h1 + 1)

#Definiendo theta o w's
Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

print(Theta1.ravel().shape)
print(Theta2.ravel().shape)


nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(72,)
(80,)
(152,)


#Funcion de activacion

In [130]:
def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))


def sigmoidGradient(z):
  
    g = np.zeros(z.shape)

    g = sigmoid(z) * (1 - sigmoid(z))

    return g

In [131]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    
    # Genera parametros randomicos ordenados en los thetas
    # for our 2 layer neural network
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))
    #Se puede aumentar para mas layers

    m = y.size
         
    J = 1
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    #Concadenamos 1s "bias"
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)
    
    #Generadmos valores 2 y concademos 1s again
    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)
    
    #Salida
    a3 = sigmoid(a2.dot(Theta2.T))
    
 
    y_matrix = y.reshape(-1)
    # print(y.shape)
    y_matrix = np.eye(num_labels)[y_matrix]
    # print(y_matrix)

    temp1 = Theta1
    temp2 = Theta2
    
    # Agregar el termino de regularización
    
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))
    
    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term
    
    # Backpropogation (En caso de tener un layer mas aumentar un delta 4)
    
    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)
    
    # Agregar regularización al gradiente(lo mismo para mas layers)

    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]
    
    # Calculamos los gradientes
    
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])
    
    return J, grad

In [132]:
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, x, y, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)
print('El costo debe esta cercano a               : 0.287629')

Costo en parametros (cargado de ex4weights): 29.848283 
El costo debe esta cercano a               : 0.287629


In [133]:
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:\n  ')
print(g)

Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [134]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):    

    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [135]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [136]:
#Estos numero son epocs
options= {'maxiter': 1000}

#  leraning rate supongo
lambda_ = 0.5

# Funcion de costo o perdida
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, x, y, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# get the solution of the optimization
nn_params = res.x
        
# Obtain Theta1 and Theta2 back from nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

In [137]:
def predict(Theta1, Theta2, X):
   
    # Useful values
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [138]:
pred = predict(Theta1, Theta2, x[:,:])
print(y)
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y[:]) * 100))

[3 0 2 3 1 1 0 0 4 0 0 0 0 1 0 1 2 1 0 6 0 0 2 1 1 0 2 2 3 4 1 0 4 0 0 4 3
 2 0 0 1 3 0 0 0 1 2 1 3 0 0 0 0 0 3 3 4 1 2 3 0 0 0 1 0 1 1 0 2 0 3 0 7 5
 0 2 0 3 1 0 0 0 0 0 0 0 0 1 0 3 2 1 3 0 1 3 2 2 1 1 1 0 0 3 0 2 1 0 2 1 1
 0 0 3 2 0 7 2 1 4 0 2 4 2 0 0 0 0 2 1 4 0 2 2 4 2 0 1 0 0 1 0 3 4 1 0 1 3
 3 0 0 1 0 5 0 0 2 0 1 2 0 0 0 0 4 1 2 1 0 1 2 0 1 1 4 2 0 0 3 0 1 0 1 0 1
 1 0 0 4 3 0 0 0 1 1 0 6 2 0 1 1 0 0 0 1 5 1 2 1 2 4 2 3 2 2 0 3 2 0 4 0 0
 0 1 1 1 2 1 1 1 0 0 2 0 1 0 0 0 1 0 0 0 3 0 1 5 3 0 2 1 0 3 1 0 0 0 4 0 1
 1 4 3 2 4 2 1 0 1 0 0 3 1 2 0 2 0 1 0 2 0 0 2 1 0 1 0 2 2 0 0 0 1 4 5 0 1
 0 1 1 0]
[3 0 2 3 1 1 0 0 4 0 0 0 0 1 0 1 2 1 0 5 0 0 2 2 1 0 2 0 3 4 1 0 4 0 0 4 1
 2 0 0 1 1 0 0 0 3 2 1 1 0 0 0 0 0 3 3 4 1 2 3 0 0 0 1 0 1 1 0 1 0 1 0 2 5
 0 2 0 3 1 0 0 0 0 0 0 0 0 1 0 1 2 1 3 0 1 1 2 2 1 1 1 0 0 3 0 2 1 0 2 1 1
 0 0 3 2 0 1 2 1 2 0 2 4 2 0 0 0 0 2 1 4 0 2 2 4 2 0 1 0 0 1 0 1 4 1 0 1 1
 1 0 0 1 0 5 0 0 2 0 1 2 0 0 0 0 4 1 2 3 0 1 2 0 1 1 4 2 0 0 1 0 1 0 1 0 1
 1 0 0 4 1 0 0 

In [139]:
pred = predict(Theta1, Theta2, xp[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == yp[:]) * 100))


[0 2 2 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1 0 0 2 0 1 1 0 0 1 0 0 0 0 0 0 2 3]
Training Set Accuracy: 69.444444


#Parte Cliente


In [160]:
#Pedir datos

mcg = float(input("Introduzca el valor MCG, Metodo McGeoch: "))
gvh = float(input("Introduzca el valor GVH, Metodo VonHeijne: "))
lab = float(input("Introduzca el valor labio, Peptidasa II: "))
chg = float(input("Introduzca el valor CHG, Presencia de carga en el extremo N de las lipoproteínas previstas: "))
aac = float(input("Introduzca el valor AAC, puntuación del análisis discriminante del contenido de aminoácidos demembrana externa y proteínas periplásmicas.: "))
alm1 = float(input("Introduzca el valor ALM1, puntuación del programa de predicción de la región que atraviesa la membrana de ALOM.: "))
alm2 = float(input("Introduzca el valor ALM2, puntuación del programa ALOM después de excluir la señal escindible putativa: "))


xprueba = np.array([[mcg, gvh, lab, chg, aac, alm1, alm1]])

pred = predict(Theta1, Theta2, xprueba)
p = pred[0]
#Resultados
if (p == 0):
  print("La ubicacion de la bacteria es en el citoplasma")
elif (p0 == 1):
  print("La ubicacion de la bacteria es en el membrana interna sin secuencia de señal")
elif (p0 == 2):
  print("La ubicacion de la bacteria es en el perisplasma")
elif (p0 == 3):
  print("La ubicacion de la bacteria es en el membrana interna, secuencia de señales que no se puede cortar")
elif (p0 == 4):
  print("La ubicacion de la bacteria es en el  membrana exterior")
elif (p0 == 5):
  print("La ubicacion de la bacteria es en el lipoproteína de la membrana externa")
elif (p0 == 6):
  print("La ubicacion de la bacteria es en el lipoproteína de la membrana interna")
elif (p0 == 7):
  print("La ubicacion de la bacteria es en el citoplasmamembrana interna, secuencia de señal escindible")

print(pred)

Introduzca el valor MCG, Metodo McGeoch: 0.5
Introduzca el valor GVH, Metodo VonHeijne: 0.48
Introduzca el valor labio, Peptidasa II: 0.76
Introduzca el valor CHG, Presencia de carga en el extremo N de las lipoproteínas previstas0.3
Introduzca el valor AAC, puntuación del análisis discriminante del contenido de aminoácidos demembrana externa y proteínas periplásmicas.0.78
Introduzca el valor ALM1, puntuación del programa de predicción de la región que atraviesa la membrana de ALOM.0.6
Introduzca el valor ALM2, puntuación del programa ALOM después de excluir la señal escindible putativa0.8
La ubicacion de la bacteria es en el membrana interna sin secuencia de señal
[1]
